---
layout: post  
---

In [ ]:
import Pkg
pkgs = [
    "DataStructures",
    "BioSequences",
    "LightGraphs",
    "PlotlyJS",
    "StatsBase",
    "GraphRecipes",
    "Plots"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Plots.plotlyjs()

TODO:

Plot the scaling of how to how to add edges to the graph based solely on the kmers present
Show graph if we assume all edges could exist
Show graph if we only list observed edges
Talk about trade-off of not being able to make connections unless they exist, which I think is the conservative and correct methodology

In [ ]:
genome = BioSequences.randdnaseq(10)

In [ ]:
k = 3

In [ ]:
kmer_counts = StatsBase.countmap(BioSequences.canonical(kmer.fw) for kmer in BioSequences.each(BioSequences.DNAMer{k}, genome))

In [ ]:
kmers = DataStructures.OrderedDict(
    kmer => (index = index, count = count) for (index, (kmer, count)) in enumerate(sort(kmer_counts))
)

In [ ]:
K = length(kmers)

In [ ]:
graph = LightGraphs.SimpleGraph(K)

In [ ]:
# # The scaling properties of looking at each kmer and seeing if any of it's neighbors are in
# # the graph is linear whilst doing a combinatorial pairwise comparison is grows exponentially
# for i in 0:31
#     n = 2^i
#     @show n
#     # if we compare each kmer to all other kmers in the list
#     @show (n-1)*(n)/2
#     # if we evaluate each kmer to see if it's neighbors are in the graph
#     @show 4*n
#     println("\n")
# end

In [ ]:
graph = LightGraphs.SimpleGraph(K)
# This may be wrong too. We shouldn't necessarily add edges just becuase they COULD exist
# Maybe we want to only add edges when they DO exist
for (index_a, kmer) in enumerate(kmers.keys)
    for neighbor in BioSequences.neighbors(kmer)
        canonical_neighbor = BioSequences.canonical(neighbor)
        index_b = get(kmers, canonical_neighbor, (index = 0, count = 0)).index
        if index_b != 0
            LightGraphs.add_edge!(graph, LightGraphs.Edge(index_a, index_b))
        end
    end
end
GraphRecipes.graphplot(graph)

In [ ]:
graph = LightGraphs.SimpleGraph(K)
for i in 1:length(genome)-k
    a_to_b_connection = genome[i:i+k]
    a = BioSequences.canonical(BioSequences.DNAMer(a_to_b_connection[1:end-1]))
    b = BioSequences.canonical(BioSequences.DNAMer(a_to_b_connection[2:end]))
    a_index = kmers[a].index
    b_index = kmers[b].index
    edge = LightGraphs.Edge(a_index, b_index)
    LightGraphs.add_edge!(graph, edge)
end
GraphRecipes.graphplot(graph)